In [49]:
# Import modules 
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords

In [28]:
# Loading the dataset
df=pd.read_csv('spam.csv',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [29]:
import os
print(os.getcwd())

C:\Users\rkt21\Codesoft\Task1_SMS_spam_detection\Task1_Handwritten_Text_Generation


In [30]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rkt21\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
# necessary columns
df=df[['v1','v2']]
# df.rename(coulmns={'v2' : 'sms', 'v1': ' names'}, inplace=True)
df = df.rename(columns={'v2':'messages' , 'v1' : 'names'})
df.head()

,names,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [40]:
# preprocessing the dataset(check for null values)
df.isnull().sum()

names       0
messages    0
dtype: int64

In [46]:
STOPWORDS = set(stopwords.words('english'))
import re
def cln_txt(text):
    text = text.lower()
    text= re.sub(r'[^0-9a-zA-Z]', ' ', text)
    text=re.sub(r'\s+', ' ',text)
    text = " ".join(word for word in text.split() if word not in STOPWORDS)
    return text

In [47]:
# cln the messages
df['cln_txt'] = df['messages'].apply(cln_txt)
df.head()

,names,messages,cln_txt
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think goes usf lives around though


In [48]:
# input split
J= df['cln_txt']
A=df['names']

In [55]:
# Model Training
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

def classi_fy(model,J,A):
    # train test split
    j_train,j_test,a_train,a_test=train_test_split(J,A,test_size=0.25, random_state=42, shuffle=True, stratify= A)
    # model training
    pipeline_model = Pipeline([('vect', CountVectorizer()),
                              ('tfidf',TfidfTransformer()),
                              ('clf', model)])
    pipeline_model.fit(j_train,a_train)
    print("Precision:", pipeline_model.score(j_test, a_test)*100)
    a_pre = pipeline_model.predict(j_test)
    print(classification_report(a_test, a_pre))

In [56]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
classi_fy(model,J,A)

Precision: 96.8413496051687
              precision    recall  f1-score   support

         ham       0.97      1.00      0.98      1206
        spam       0.99      0.77      0.87       187

    accuracy                           0.97      1393
   macro avg       0.98      0.88      0.92      1393
weighted avg       0.97      0.97      0.97      1393



In [57]:
from sklearn.naive_bayes import MultinomialNB
model= MultinomialNB()
classi_fy(model,J,A)

Precision: 96.69777458722182
              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1206
        spam       1.00      0.75      0.86       187

    accuracy                           0.97      1393
   macro avg       0.98      0.88      0.92      1393
weighted avg       0.97      0.97      0.96      1393



In [61]:
from sklearn.svm import SVC
model = SVC(C=7)
classi_fy(model,J,A)

Precision: 98.27709978463747
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      1206
        spam       1.00      0.87      0.93       187

    accuracy                           0.98      1393
   macro avg       0.99      0.94      0.96      1393
weighted avg       0.98      0.98      0.98      1393

